In [2]:
# import the necessary packages
import mysql.connector
from mysql.connector import Error
import pandas as pd
import os
import time

In [18]:
# 連接資料庫
connection = mysql.connector.connect(
    host='localhost',  # 主機名稱
    database='example',  # 資料庫名稱
    user='root',  # 帳號
    password='root'
)  # 密碼
if connection.is_connected():
    # 顯示資料庫版本
    db_Info = connection.get_server_info()
    print("資料庫版本：", db_Info)

    # 顯示目前使用的資料庫
    cur = connection.cursor()
    cur.execute("SELECT DATABASE();")
    record = cur.fetchone()
    print("目前使用的資料庫：", record)

資料庫版本： 5.5.5-10.11.2-MariaDB
目前使用的資料庫： ('example',)


In [4]:
#讀取資料
Data= os.path.join('fake_data.csv') #原始資料
df = pd.read_csv(Data)
df.head()

,id,name,gender,age,height_cm,weight_kg,country,position
0,1,Amy,0,18,160,48,Taiwan,Student
1,2,John,1,25,175,75,United States,Programmer
2,3,Sarah,0,33,168,62,Canada,Doctor
3,4,David,1,42,180,85,Australia,Engineer
4,5,Emily,0,29,162,52,United Kingdom,Teacher


In [5]:
df.dtypes

id            int64
name         object
gender        int64
age           int64
height_cm     int64
weight_kg     int64
country      object
position     object
dtype: object

In [6]:
df.columns

Index(['id', 'name', 'gender', 'age', 'height_cm', 'weight_kg', 'country',
       'position'],
      dtype='object')

In [66]:
#transform datatype (in a simple way)
#for col in df.select_dtypes(include=['int64']):
#    df[col] = df[col].astype('int32')

In [7]:
#transform datatype
df = df.astype('str')

In [13]:
#Insert
t0 = time.time()
columns = ','.join([f'{x}' for x in df.columns])
param_placeholders = ','.join(['%s' for x in range(len(df.columns))])

def insert(c0, c1, c2, c3, c4, c5, c6, c7):
    try:
        insertStatement = f"INSERT INTO profile ({columns}) VALUES ({param_placeholders})"
        data = (c0, c1, c2, c3, c4, c5, c6, c7)
        cur.execute(insertStatement, data)
        connection.commit()
        print(time.time()-t0)
        print("Successfully added entry to database")
    except Error as e:
        print(f"Error adding entry to database: {e}")

col_m = df.columns.to_list()
for i in range(len(df)):
    insert(df[col_m[0]][i], df[col_m[1]][i], df[col_m[2]][i], df[col_m[3]][i], df[col_m[4]][i], df[col_m[5]][i], df[col_m[6]][i], df[col_m[7]][i])

0.0020003318786621094
Successfully added entry to database
0.003998517990112305
Successfully added entry to database
0.005000114440917969
Successfully added entry to database
0.006001710891723633
Successfully added entry to database
0.007000923156738281
Successfully added entry to database
0.008000850677490234
Successfully added entry to database
0.008000850677490234
Successfully added entry to database
0.009000301361083984
Successfully added entry to database
0.010000467300415039
Successfully added entry to database
0.010998249053955078
Successfully added entry to database
0.011998891830444336
Successfully added entry to database
0.012998819351196289
Successfully added entry to database
0.014000415802001953
Successfully added entry to database
0.014000415802001953
Successfully added entry to database
0.014998912811279297
Successfully added entry to database
0.01599907875061035
Successfully added entry to database
0.01599907875061035
Successfully added entry to database
0.0169990062713

In [10]:
# Insert (精簡版)
t0 = time.time()
columns = ','.join([f'{x}' for x in df.columns])
param_placeholders = ','.join(['%s' for x in range(len(df.columns))])

def insert(*args):
    try:
        insertStatement = f"INSERT INTO profile ({columns}) VALUES ({param_placeholders})"
        cur.execute(insertStatement, args)
        connection.commit()
        print(time.time()-t0)
        print("Successfully added entry to database")
    except Error as e:
        print(f"Error adding entry to database: {e}")

for i in range(len(df)):
    insert(*df.iloc[i])

0.0019981861114501953
Successfully added entry to database
0.002997875213623047
Successfully added entry to database
0.003999233245849609
Successfully added entry to database
0.004998207092285156
Successfully added entry to database
0.006000041961669922
Successfully added entry to database
0.006998777389526367
Successfully added entry to database
0.008998394012451172
Successfully added entry to database
0.008998394012451172
Successfully added entry to database
0.009997367858886719
Successfully added entry to database
0.010996818542480469
Successfully added entry to database
0.01199650764465332
Successfully added entry to database
0.01199650764465332
Successfully added entry to database
0.012996673583984375
Successfully added entry to database
0.013999223709106445
Successfully added entry to database
0.014998435974121094
Successfully added entry to database
0.014998435974121094
Successfully added entry to database
0.015999317169189453
Successfully added entry to database
0.0169994831085

In [16]:
# Update
columns = df.columns.tolist()
param_placeholders = ','.join([f'{col}=%s' for col in columns if col != 'id'])


## 將id作為primary key以更新其餘欄位 (注意最後的%s為id，因此所對應c7的部分也應為id欄位)
def update(c0, c1, c2, c3, c4, c5, c6, c7):
    try:
        updateStatement = f"UPDATE profile SET {param_placeholders} WHERE id = %s"
        data = (c0, c1, c2, c3, c4, c5, c6, c7)
        cur.execute(updateStatement, data)
        connection.commit()
        print(time.time()-t0)
        print("Successfully updated entry to database")
    except Error as e:
        print(f"Error updating entry to database: {e}")
        
t0 = time.time()
for index, row in df.iterrows():
    update(row['name'], row['gender'], row['age'], row['height_cm'], row['weight_kg'], row['country'], row['position'], row['id'])

0.002000570297241211
Successfully updated entry to database
0.0030007362365722656
Successfully updated entry to database
0.004002809524536133
Successfully updated entry to database
0.004002809524536133
Successfully updated entry to database
0.005001068115234375
Successfully updated entry to database
0.005001068115234375
Successfully updated entry to database
0.0060002803802490234
Successfully updated entry to database
0.0060002803802490234
Successfully updated entry to database
0.007001399993896484
Successfully updated entry to database
0.007001399993896484
Successfully updated entry to database
0.007001399993896484
Successfully updated entry to database
0.00800013542175293
Successfully updated entry to database
0.00800013542175293
Successfully updated entry to database
0.009000301361083984
Successfully updated entry to database
0.009000301361083984
Successfully updated entry to database
0.009000301361083984
Successfully updated entry to database
0.010000467300415039
Successfully updat

In [19]:
# Update (精簡版)
columns = df.columns.tolist()
param_placeholders = ','.join([f'{col}=%s' for col in columns if col != 'id'])

def update(row):
    try:
        updateStatement = f"UPDATE profile SET {param_placeholders} WHERE id = %s"
        data = tuple(row[col] for col in columns if col != 'id') + (row['id'],)
        cur.execute(updateStatement, data)
        connection.commit()
        print(time.time()-t0)
        print("Successfully updated entry to database")
    except Error as e:
        print(f"Error updating entry to database: {e}")

t0 = time.time()
for index, row in df.iterrows():
    update(row)

0.0009999275207519531
Successfully updated entry to database
0.0029993057250976562
Successfully updated entry to database
0.0039980411529541016
Successfully updated entry to database
0.0039980411529541016
Successfully updated entry to database
0.005000114440917969
Successfully updated entry to database
0.005000114440917969
Successfully updated entry to database
0.006000518798828125
Successfully updated entry to database
0.006000518798828125
Successfully updated entry to database
0.006000518798828125
Successfully updated entry to database
0.00699925422668457
Successfully updated entry to database
0.00699925422668457
Successfully updated entry to database
0.008000850677490234
Successfully updated entry to database
0.008999109268188477
Successfully updated entry to database
0.008999109268188477
Successfully updated entry to database
0.008999109268188477
Successfully updated entry to database
0.009999513626098633
Successfully updated entry to database
0.009999513626098633
Successfully upda

In [17]:
if connection.is_connected():
    # close the database connection
    connection.close()
    print("資料庫連線已關閉。")
else:
    print("資料庫連線未開啟。")

資料庫連線已關閉。
